<a href="https://colab.research.google.com/github/move-coop/api/blob/master/66Degree_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Atnafu Dargaso, Sr. Data Engineer

---

# Supermarket Sales Data Analysis

In this project, I analyze supermarket sales data by performing data extraction, transformation, and loading (ETL) operations. I load the transformed data into an SQLite database, create an analytical report, and visualize the results using Plotly for an interactive presentation.

---

## Table of Contents
1. [Environment Setup and Data Preparation](#section-0)
2. [Data Extraction, Transformation, and Loading (ETL)](#section-1)
3. [SQL Table Creation](#section-2)
4. [Report Generation](#section-3)
5. [Visualization with Plotly](#section-4)

---

<a id="section-0"></a>
## 1. Environment Setup and Data Preparation

### Description
To execute this project in Google Colab, I first install necessary libraries and download the dataset from Kaggle. This setup includes configuring the Kaggle API to enable access to the dataset.

### Prerequisites
For this setup, I run the following code to install and configure the Kaggle API, download the dataset, and prepare it for analysis in Colab. After running these commands, the dataset is saved as `supermarket_sales.csv` and ready for further processing.

In [46]:
# Install required libraries
!pip install kaggle pandas plotly

# Upload kaggle.json for API access
from google.colab import files
files.upload()

# Configure Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d aungpyaeap/supermarket-sales -p /content
!unzip -o /content/supermarket-sales.zip -d /content
!mv /content/supermarket_sales*.csv /content/supermarket_sales.csv

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales
License(s): other
supermarket-sales.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/supermarket-sales.zip
  inflating: /content/supermarket_sales - Sheet1.csv  
mv: target '/content/supermarket_sales.csv' is not a directory


---

<a id="section-1"></a>
## 2. Data Extraction, Transformation, and Loading (ETL)

### Description
I load the supermarket sales data and transform it into three structured tables to support efficient analysis. The tables created are:
- **Customer Dimension Table**: Contains unique customer information, including ID, gender, and customer type.
- **Product Dimension Table**: Contains unique product lines and their unit prices.
- **Sales Fact Table**: Contains transactional sales data, including invoice ID, customer ID, product line, quantity, date, total, and rating.

In [47]:
import pandas as pd
import sqlite3

# Load the dataset
data = pd.read_csv('/content/supermarket_sales.csv')

# Create Customer Dimension Table
customer_dim = data[['Customer type', 'Gender', 'Customer type']].drop_duplicates()
customer_dim.columns = ['customer_id', 'gender', 'customer_type']
customer_dim = customer_dim.drop_duplicates(subset='customer_id')

# Display Customer Dimension Table
print("\nCustomer Dimension Table:")
display(customer_dim)

# Create Product Dimension Table
product_dim = data[['Product line', 'Unit price']].drop_duplicates()
product_dim.columns = ['product_line', 'unit_price']
product_dim = product_dim.drop_duplicates(subset='product_line')

# Display Product Dimension Table
print("\nProduct Dimension Table:")
display(product_dim)

# Create Sales Fact Table
sales_fact = data[['Invoice ID', 'Customer type', 'Product line', 'Quantity', 'Date', 'Total', 'Rating']]
sales_fact.columns = ['invoice_id', 'customer_id', 'product_line', 'quantity', 'date', 'total', 'rating']

# Display Sales Fact Table
print("\nSales Fact Table:")
display(sales_fact)


Customer Dimension Table:


,customer_id,gender,customer_type
0,Member,Female,Member
1,Normal,Female,Normal



Product Dimension Table:


,product_line,unit_price
0,Health and beauty,74.69
1,Electronic accessories,15.28
2,Home and lifestyle,46.33
4,Sports and travel,86.31
9,Food and beverages,54.84
10,Fashion accessories,14.48



Sales Fact Table:


,invoice_id,customer_id,product_line,quantity,date,total,rating
0,750-67-8428,Member,Health and beauty,7,1/5/2019,548.9715,9.1
1,226-31-3081,Normal,Electronic accessories,5,3/8/2019,80.2200,9.6
2,631-41-3108,Normal,Home and lifestyle,7,3/3/2019,340.5255,7.4
3,123-19-1176,Member,Health and beauty,8,1/27/2019,489.0480,8.4
4,373-73-7910,Normal,Sports and travel,7,2/8/2019,634.3785,5.3
...,...,...,...,...,...,...,...
995,233-67-5758,Normal,Health and beauty,1,1/29/2019,42.3675,6.2
996,303-96-2227,Normal,Home and lifestyle,10,3/2/2019,1022.4900,4.4
997,727-02-1313,Member,Food and beverages,1,2/9/2019,33.4320,7.7
998,347-56-2442,Normal,Home and lifestyle,1,2/22/2019,69.1110,4.1


---

<a id="section-2"></a>
## 3. SQL Table Creation

### Description
Using an SQLite in-memory database within Colab, I create tables to store the transformed data. The tables include **Customer**, **Product**, and **Sales** to facilitate efficient querying and reporting.



In [48]:

# Connect to SQLite (in-memory for testing)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE Customer (customer_id TEXT PRIMARY KEY, gender TEXT, customer_type TEXT)''')
cursor.execute('''CREATE TABLE Product (product_line TEXT PRIMARY KEY, unit_price REAL)''')
cursor.execute('''CREATE TABLE Sales (invoice_id TEXT PRIMARY KEY, customer_id TEXT, product_line TEXT, quantity INTEGER, date TEXT, total REAL, rating REAL)''')

# Insert data into tables
customer_dim.to_sql('Customer', conn, if_exists='append', index=False)
product_dim.to_sql('Product', conn, if_exists='append', index=False)
sales_fact.to_sql('Sales', conn, if_exists='append', index=False)

1000



---

<a id="section-3"></a>
## 4. Report Generation

### Description
I generate an analytical report to analyze total sales and average ratings for each product line. Using SQL, I calculate total sales and average rating per product line and sort the results by total sales in descending order.

In [49]:
query = '''
    SELECT
        P.product_line,
        SUM(S.total) AS total_sales,
        AVG(S.rating) AS avg_rating
    FROM
        Sales S
        JOIN Product P ON S.product_line = P.product_line
    GROUP BY
        P.product_line
    ORDER BY
        total_sales DESC
'''

# Execute the query and load the result into a DataFrame
report = pd.read_sql_query(query, conn)
print("\nAnalytical Report:")
display(report)


Analytical Report:


,product_line,total_sales,avg_rating
0,Food and beverages,56144.8440,7.113218
1,Sports and travel,55122.8265,6.916265
2,Electronic accessories,54337.5315,6.924706
3,Fashion accessories,54305.8950,7.029213
4,Home and lifestyle,53861.9130,6.837500
5,Health and beauty,49193.7390,7.003289




---

<a id="section-4"></a>
## 5. Visualization with Plotly

### Description
To make the report more interactive and insightful, I use Plotly to create visualizations. These include:
- A bar chart showing total sales by product line.
- A line chart with markers for the average rating by product line.



In [50]:
import plotly.express as px
import plotly.graph_objects as go

# Bar chart for total sales by product line
fig_sales = px.bar(
    report,
    x='product_line',
    y='total_sales',
    title='Total Sales by Product Line',
    labels={'product_line': 'Product Line', 'total_sales': 'Total Sales'},
    text_auto=True
)
fig_sales.update_layout(xaxis_title='Product Line', yaxis_title='Total Sales')
fig_sales.show()

# Scatter plot for average rating by product line
fig_rating = go.Figure()
fig_rating.add_trace(go.Scatter(
    x=report['product_line'],
    y=report['avg_rating'],
    mode='markers+lines',
    marker=dict(size=10, color='red'),
    name='Average Rating'
))
fig_rating.update_layout(
    title='Average Rating by Product Line',
    xaxis_title='Product Line',
    yaxis_title='Average Rating',
    yaxis=dict(range=[0, 10])  # Assuming rating scale is 0-20
)
fig_rating.show()



---

## Closing the Database Connection
I close the database connection to free up resources after completing the analysis.

In [51]:
# Closing the Connection
conn.close()

---

## Conclusion
In this project, I demonstrate my ability to set up an ETL pipeline, load data into an SQLite database, generate SQL-based analytical reports, and create interactive visualizations in Google Colab. This process provides valuable insights into sales trends and customer preferences, which can help drive business decisions.

### Key Insights
- **Top Sellers**:
  - "Food and beverages" leads in sales ($56,144.84), followed by "Sports and travel" and "Electronic accessories."
  
- **Customer Satisfaction**:
  - High ratings for "Fashion accessories" (7.03) and "Health and beauty" (7.00).
  - "Home and lifestyle" has the lowest satisfaction rating (6.84).

- **Sales vs. Satisfaction**:
  - High sales don’t always mean high satisfaction; categories like "Health and beauty" have lower sales but strong ratings.

### Action Items
1. **Invest in High Sellers**: Prioritize "Food and beverages" to sustain revenue.
2. **Improve "Home and Lifestyle"**: Enhance quality or adjust pricing to boost satisfaction.
3. **Promote High-Rated Categories**: Market "Health and beauty" to leverage customer approval.
